# Data Processing and EDA using Pyspark

## Data from trusted to Refine

## load data from trusted

In [1]:
#%pip install pyarrow

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1709866115604_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
path_trusted_source_1='s3://trabajo1datalake/trusted/climatedata_imf_org/level_sea'
# Load raw data
df_trusted_source_1=spark.read.parquet(path_trusted_source_1,
                  inferSchema=True,
                  header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
path_trusted_source_1_1='s3://trabajo1datalake/trusted/climatedata_imf_org/dioxido_carbono'
    # Load raw data
df_trusted_source_1_1=spark.read.parquet(path_trusted_source_1_1,
                  inferSchema=True,
                  header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
path_trusted_source_2='s3://trabajo1datalake/trusted/ourworldindata_org/owid-co2-data'
# Load raw data
df_trusted_source_2=spark.read.parquet(path_trusted_source_2,
                  inferSchema=True,
                  header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#printSchema
df_trusted_source_1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ObjectId: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- ISO2: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- CTS_Code: string (nullable = true)
 |-- CTS_Name: string (nullable = true)
 |-- CTS_Full_Descriptor: string (nullable = true)
 |-- Measure: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Value: double (nullable = true)

In [6]:
#select columns
df_refine_source_1 = df_trusted_source_1.select('Country','Indicator','Unit','Date','Value')
df_refine_source_1.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-----------+----------+------+
|Country|           Indicator|       Unit|      Date| Value|
+-------+--------------------+-----------+----------+------+
|  World|Change_in_mean_se...|Millimeters|1994-06-05|-12.63|
|  World|Change_in_mean_se...|Millimeters|1994-09-02| -2.63|
|  World|Change_in_mean_se...|Millimeters|1994-10-22|  7.49|
|  World|Change_in_mean_se...|Millimeters|1994-12-20|-18.55|
|  World|Change_in_mean_se...|Millimeters|1998-04-13| -78.0|
+-------+--------------------+-----------+----------+------+
only showing top 5 rows

In [7]:
df_refine_source_1_1.select('Unit').distinct().collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_refine_source_1_1' is not defined
Traceback (most recent call last):
NameError: name 'df_refine_source_1_1' is not defined



In [8]:
df_refine_source_1.select('Indicator').distinct().collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Indicator='Change_in_mean_sea_level_Sea_level_Trend'), Row(Indicator='Change_in_mean_sea_level_Sea_level_TOPEXPoseidon'), Row(Indicator='Change_in_mean_sea_level_Sea_level_Jason1'), Row(Indicator='Change_in_mean_sea_level_Sea_level_Jason2'), Row(Indicator='Change_in_mean_sea_level_Sea_level_Jason3')]

In [9]:
from pyspark.sql.functions import year, month
# Extraer el año y el mes de la columna 'Date'
df_refine_source_1 = df_refine_source_1.withColumn("Year", year("Date")).withColumn("Month", month("Date"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Agrupar por año y mes y calcular la media de 'Value'
df_refine_grouped_source_1 = df_refine_source_1.groupBy("Year", "Month").pivot("Indicator").avg("Value").orderBy("Year", "Month")


df_refine_grouped_source_1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-----------------------------------------+-----------------------------------------+-----------------------------------------+------------------------------------------------+----------------------------------------+
|Year|Month|Change_in_mean_sea_level_Sea_level_Jason1|Change_in_mean_sea_level_Sea_level_Jason2|Change_in_mean_sea_level_Sea_level_Jason3|Change_in_mean_sea_level_Sea_level_TOPEXPoseidon|Change_in_mean_sea_level_Sea_level_Trend|
+----+-----+-----------------------------------------+-----------------------------------------+-----------------------------------------+------------------------------------------------+----------------------------------------+
|1992|   12|                                     null|                                     null|                                     null|                               4.265600000000001|                                    null|
|1993|    1|                                     null|                              

In [11]:
#info about dataframe
df_refine_grouped_source_1.describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+------------------------------------------------+----------------------------------------+
|summary|              Year|             Month|Change_in_mean_sea_level_Sea_level_Jason1|Change_in_mean_sea_level_Sea_level_Jason2|Change_in_mean_sea_level_Sea_level_Jason3|Change_in_mean_sea_level_Sea_level_TOPEXPoseidon|Change_in_mean_sea_level_Sea_level_Trend|
+-------+------------------+------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+------------------------------------------------+----------------------------------------+
|  count|               360|               360|                                      137|                                      107|                                       82|                                   

### lo mismo para la fuente 1.1

In [12]:
#printSchema
df_trusted_source_1_1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ObjectId: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- ISO2: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- CTS_Code: string (nullable = true)
 |-- CTS_Name: string (nullable = true)
 |-- CTS_Full_Descriptor: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Value: double (nullable = true)

In [13]:
#select columns
df_refine_source_1_1 = df_trusted_source_1_1.select('Country','Indicator','Unit','Date','Value')
df_refine_source_1_1.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-----------------+----------+------+
|Country|           Indicator|             Unit|      Date| Value|
+-------+--------------------+-----------------+----------+------+
|  World|Monthly_Atmospher...|          Percent|1969-10-01|  0.48|
|  World|Monthly_Atmospher...|          Percent|1968-10-01|  0.27|
|  World|Monthly_Atmospher...|Parts_Per_Million|1992-07-01|357.15|
|  World|Monthly_Atmospher...|          Percent|2023-10-01|  0.74|
|  World|Monthly_Atmospher...|Parts_Per_Million|1978-08-01|334.42|
+-------+--------------------+-----------------+----------+------+
only showing top 5 rows

In [14]:
df_refine_source_1_1.select('Indicator').distinct().collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Indicator='Monthly_Atmospheric_Carbon_Dioxide_Concentrations_Year_on_Year_Percentage_Change'), Row(Indicator='Monthly_Atmospheric_Carbon_Dioxide_Concentrations')]

In [15]:
from pyspark.sql.functions import year, month
# Extraer el año y el mes de la columna 'Date'
df_refine_source_1_1 = df_refine_source_1_1.withColumn("Year", year("Date")).withColumn("Month", month("Date"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Agrupar por año y mes y calcular la media de 'Value'
df_refine_grouped_source_1_1 = df_refine_source_1_1.groupBy("Year", "Month").pivot("Indicator").avg("Value").orderBy("Year", "Month")


df_refine_grouped_source_1_1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-------------------------------------------------+--------------------------------------------------------------------------------+
|Year|Month|Monthly_Atmospheric_Carbon_Dioxide_Concentrations|Monthly_Atmospheric_Carbon_Dioxide_Concentrations_Year_on_Year_Percentage_Change|
+----+-----+-------------------------------------------------+--------------------------------------------------------------------------------+
|1958|    3|                                            315.7|                                                                            null|
|1958|    4|                                           317.45|                                                                            null|
|1958|    5|                                           317.51|                                                                            null|
|1958|    6|                                           317.24|                                                                          

In [17]:
#info about dataframe
df_refine_grouped_source_1_1.describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+-----------------+-------------------------------------------------+--------------------------------------------------------------------------------+
|summary|              Year|            Month|Monthly_Atmospheric_Carbon_Dioxide_Concentrations|Monthly_Atmospheric_Carbon_Dioxide_Concentrations_Year_on_Year_Percentage_Change|
+-------+------------------+-----------------+-------------------------------------------------+--------------------------------------------------------------------------------+
|  count|               791|              791|                                              791|                                                                             779|
|   mean|1990.6245259165614|6.505689001264223|                                358.2500505688998|                                                             0.44745827984595515|
| stddev|19.041177371494015|3.452704755107532|                                33.81632708794604|              

### hacer join de las dos tablas por año mes

In [18]:
joined_df = df_refine_grouped_source_1_1.join(df_refine_grouped_source_1, ['Year', 'Month'], 'inner')

joined_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-------------------------------------------------+--------------------------------------------------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+------------------------------------------------+----------------------------------------+
|Year|Month|Monthly_Atmospheric_Carbon_Dioxide_Concentrations|Monthly_Atmospheric_Carbon_Dioxide_Concentrations_Year_on_Year_Percentage_Change|Change_in_mean_sea_level_Sea_level_Jason1|Change_in_mean_sea_level_Sea_level_Jason2|Change_in_mean_sea_level_Sea_level_Jason3|Change_in_mean_sea_level_Sea_level_TOPEXPoseidon|Change_in_mean_sea_level_Sea_level_Trend|
+----+-----+-------------------------------------------------+--------------------------------------------------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------

In [19]:
#printSchema
joined_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Monthly_Atmospheric_Carbon_Dioxide_Concentrations: double (nullable = true)
 |-- Monthly_Atmospheric_Carbon_Dioxide_Concentrations_Year_on_Year_Percentage_Change: double (nullable = true)
 |-- Change_in_mean_sea_level_Sea_level_Jason1: double (nullable = true)
 |-- Change_in_mean_sea_level_Sea_level_Jason2: double (nullable = true)
 |-- Change_in_mean_sea_level_Sea_level_Jason3: double (nullable = true)
 |-- Change_in_mean_sea_level_Sea_level_TOPEXPoseidon: double (nullable = true)
 |-- Change_in_mean_sea_level_Sea_level_Trend: double (nullable = true)

In [20]:
new_df =joined_df.select('Year','Month','Monthly_Atmospheric_Carbon_Dioxide_Concentrations','Change_in_mean_sea_level_Sea_level_Jason1')
new_df.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-------------------------------------------------+-----------------------------------------+
|Year|Month|Monthly_Atmospheric_Carbon_Dioxide_Concentrations|Change_in_mean_sea_level_Sea_level_Jason1|
+----+-----+-------------------------------------------------+-----------------------------------------+
|1997|   11|                                           362.65|                                     null|
+----+-----+-------------------------------------------------+-----------------------------------------+
only showing top 1 row

In [21]:
new_df.describe('Monthly_Atmospheric_Carbon_Dioxide_Concentrations').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------------------------------------+
|summary|Monthly_Atmospheric_Carbon_Dioxide_Concentrations|
+-------+-------------------------------------------------+
|  count|                                              360|
|   mean|                               385.82861111111083|
| stddev|                               18.451715632015866|
|    min|                                           354.14|
|    max|                                           420.97|
+-------+-------------------------------------------------+

In [22]:
#filter the records
new_df.filter(new_df['Month']==12).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-------------------------------------------------+-----------------------------------------+
|Year|Month|Monthly_Atmospheric_Carbon_Dioxide_Concentrations|Change_in_mean_sea_level_Sea_level_Jason1|
+----+-----+-------------------------------------------------+-----------------------------------------+
|1995|   12|                                           360.82|                                     null|
|2010|   12|                                           389.99|                        42.71204081632651|
|1994|   12|                                            359.1|                                     null|
|2006|   12|                                           382.02|                        27.77723684210527|
|2014|   12|                                           399.08|                                     null|
|2013|   12|                                           397.03|                                     null|
|2011|   12|                                           

In [23]:
#filter the multiple conditions
new_df.filter(new_df['Month']==12).filter(new_df['Monthly_Atmospheric_Carbon_Dioxide_Concentrations'] >360).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-------------------------------------------------+-----------------------------------------+
|Year|Month|Monthly_Atmospheric_Carbon_Dioxide_Concentrations|Change_in_mean_sea_level_Sea_level_Jason1|
+----+-----+-------------------------------------------------+-----------------------------------------+
|1995|   12|                                           360.82|                                     null|
|2010|   12|                                           389.99|                        42.71204081632651|
|2006|   12|                                           382.02|                        27.77723684210527|
|2014|   12|                                           399.08|                                     null|
|2013|   12|                                           397.03|                                     null|
|2011|   12|                                           392.06|                                  60.1664|
|2017|   12|                                           

In [24]:
#filter the multiple conditions
new_df.filter((new_df['Monthly_Atmospheric_Carbon_Dioxide_Concentrations']>400)&(new_df['Change_in_mean_sea_level_Sea_level_Jason1'] >19.1456)).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-------------------------------------------------+-----------------------------------------+
|Year|Month|Monthly_Atmospheric_Carbon_Dioxide_Concentrations|Change_in_mean_sea_level_Sea_level_Jason1|
+----+-----+-------------------------------------------------+-----------------------------------------+
|2013|    5|                                           400.02|                       32.053066666666666|
+----+-----+-------------------------------------------------+-----------------------------------------+

In [25]:
# UDF
from pyspark.sql.functions import udf


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
#normal function
def co2_range(co2_concentration):
    if co2_concentration >= 405:
        return 'High'
    elif co2_concentration < 405 and co2_concentration >= 380:
        return 'Mid'
    else:
        return 'Low'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
from pyspark.sql.types import StringType
#create udf using python function
brand_udf=udf(co2_range,StringType())
#apply udf on dataframe
new_df = new_df.withColumn('co2_range',brand_udf(new_df['Monthly_Atmospheric_Carbon_Dioxide_Concentrations']))
new_df.show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-------------------------------------------------+-----------------------------------------+---------+
|Year|Month|Monthly_Atmospheric_Carbon_Dioxide_Concentrations|Change_in_mean_sea_level_Sea_level_Jason1|co2_range|
+----+-----+-------------------------------------------------+-----------------------------------------+---------+
|1997|11   |362.65                                           |null                                     |Low      |
|2022|10   |415.74                                           |null                                     |High     |
|2005|5    |382.64                                           |-12.792266666666665                      |Mid      |
|1995|12   |360.82                                           |null                                     |Low      |
|1998|7    |367.98                                           |null                                     |Low      |
|2010|12   |389.99                                           |42.71204081632651 

In [28]:
#target directory
pathcsv_out='/user/hadoop/df_co2_level_sea'

# hacia S3
pathcsv_out_s3='s3://trabajo1datalake/refined/df_co2_level_sea'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
#save the dataframe as single csv
new_df.coalesce(1).write.mode('overwrite').format("csv").option("header","true").save(pathcsv_out)
new_df.coalesce(1).write.mode('overwrite').format("csv").option("header","true").save(pathcsv_out_s3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
#target location
pathparquet_out='/user/hadoop/co2_and_level_sea'

# hacia S3
write_uri='s3://trabajo1datalake/refined/co2_and_level_sea'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
#save the data into parquet format
joined_df.write.mode('overwrite').format('parquet').save(pathparquet_out)
joined_df.write.mode('overwrite').format('parquet').save(write_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…